<div style='width: auto; float: center'/>
<table align="center">
    <tr>
        <th>
                <img src="https://ingenieria.unal.edu.co/pri/images/catedra_header_2021.png" style='height: 175px; width: auto; object-fit: contain;float: center'/>                                		
        </th>     
    </tr>
</table>

</div>

<div >
  <h1 align="center">  Asset Management Principles For Modern Power Systems    </h1>  
  <h2 align="center">  Unit 1: Criticallity Assessment     </h2>    
			<h3 align="center">    David L. Alvarez A, Ph.D    </h3>
			<h4 align="center">    dlalvareza@unal.edu.co      </h4>
            <h4 align="center">    Copyright (c) 2021 dlalvareza    </h4>
</div>	

<div style='width: auto; float: center'/>
<table align="center">
    <tr>
        <th>
       <img src="https://www.uqtr.ca/u20-img/logos/logo-uqtr-entete.png" style='height: 75px; width: auto; object-fit: contain;float: right'/>                                	
        </th>                 
        <th>
                <img src="https://oraprdnt.uqtr.uquebec.ca/pls/public/docs/GSC6627/O0003430433_GREI_LOGO_court.PNG" style='height: 75px; width: auto; object-fit: contain;float: right'/>                                		
        </th>  
        <th>
                <img src="https://oraprdnt.uqtr.uquebec.ca/pls/public/docs/GSC6627/O0003430597_MRIF_w3_couleur.png" style='height: 75px; width: auto; object-fit: contain;float: right'/>                                		
        </th>  
        <th>
       <img src="../STATIC/EMC_ACI.svg" style='height: 50px; width: auto; object-fit: contain;float: right'/>                                	
        </th>                 
        <th>
                <img src="https://unal.edu.co/typo3conf/ext/unal_skin_default/Resources/Public/images/escudoUnal_black.png" style='height: 75px; width: auto; object-fit: contain;float: right'/>                                		
        </th>          
    </tr>
</table>    

## Introduction
Whit this notebook, the operational criticality assessment is performed. The analysis is carried out by assessing the impact on energy not supplied as well as the operational cost associated to a physical asset failure.  

## CIGRE network benchmark DER in Medium Voltage Systems
![title](../STATIC/01_CIGRE_MV_Distribution_DER_Assets_Criticality.svg)

### Neccesary libraries

In [1]:
import sys
import datetime 
import pandas as pd
import calendar
from math import sqrt
from ipywidgets import interact
from ipywidgets import fixed
import plotly.express as px
from plotly.subplots import make_subplots

### Load PywerAPM libraries

In [2]:
sys.path.insert(0,'../CASES/05_Challenge_Data/')
sys.path.insert(0,'../APM/BIN/')
from ST_AM_Contingencies_Analysis import Real_Time_Contingencies as Cont_Assessment
from ST_AM_Contingencies_Ploty import  Plot_Stack_By_Day, Plot_Security_Opteration
from ACM_Module import ACM

# 1. Normal operataion 

### Asset fleet
<div style='width: auto; float: center; background-color: black'/>
                <img src="../STATIC/01_CIGRE_MV_Distribution_DER_Assets.svg" style='height: 400px; width: auto; object-fit: contain;float: center'/>                 </table>

## 1.1 Prepare case settings

In [3]:
from PywerAPM_Case_Setting import*                         # Import case settings
Cont_A = Cont_Assessment(case_settings,pp_case='json')     # Load network benchmark and contingencies network 
Cont_A.Load_Growth_Update(load_growth)                     # Update load growth 

## 1.2 Assessment of the network durign normal operation

In [4]:
df_base_line,df_load,df_sec = Cont_A.Run_Case_Load_growth(Cont_A.net,load_growth)
S_base = sqrt((Cont_A.net.load['p_mw']**2+  Cont_A.net.load['q_mvar']**2).max())

## 1.3 Plot load profiles by week day during 24 h

In [5]:
def plot_load_profile(day_name):
    fig = Plot_Stack_By_Day(df_load,day_name)

In [16]:
day_list = list(calendar.day_name)
interact(plot_load_profile, day_name=day_list);

interactive(children=(Dropdown(description='day_name', options=('Monday', 'Tuesday', 'Wednesday', 'Thursday', …

# 2.0 Criticality assessment 

In [7]:
def plot_security_profile_type_case(case):
    interact(plot_security_profile_type, day_name=day_list,case=fixed(case))

def choose_assessment(day_name,df):
    l_asset = ['TR','BU','LN']
    interact(plot_security_profile,l_type=l_asset,day_name=fixed(day_name),df = fixed(df))
def plot_security_profile_type(day_name,case):
    if case == 'Base':
        df=df_base_line
        choose_assessment(day_name,df)
    elif case == 'Contingency':
        asset_type_list = Cont_A.asset_list['Type'].unique()
        interact(Asset_Selection, asset_type=asset_type_list,day=fixed(day_name))   
        
def Asset_Selection(asset_type,day):
    asset_id_list                = Cont_A.asset_list[Cont_A.asset_list['Type']==asset_type].index.values
    interact(disc_asset, asset_id=asset_id_list,day=fixed(day))  

def disc_asset(asset_id,day):
    asset_fail                        = asset_id
    net_lf                            = Cont_A.Net_Configurarion_during_Contingency(Cont_A.net,asset_fail)
    df_base_C,df_load_C,sec_assesment = Cont_A.Run_Case_Load_growth(net_lf,load_growth,day=[day])
    choose_assessment(day,df_base_C)
    hour_list = df_base_C[df_base_C.Day==day].Hour.unique()
    sec_assesment_by_day = sec_assesment['cont'][day]
    interact(df_security_violation_by_hour, hour=hour_list,df_sec=fixed(sec_assesment_by_day))  
def df_security_violation_by_hour(hour,df_sec):
    df = df_sec[hour]
    if df.empty:
        print('None security violations')
    else:
        print(df)        
def plot_security_profile(l_type,day_name,df):
    TR = False
    LN = False
    BU = False
    if l_type=='TR':
        TR  = True
    elif l_type=='LN':
        LN  = True
    elif l_type=='BU':
        BU  = True
    Plot_Security_Opteration(df,day_name,S_base,TR,LN,BU)

## 2.1 Security assesment

In [17]:
l_case = ['Base','Contingency']
interact(plot_security_profile_type_case,case=l_case);

interactive(children=(Dropdown(description='case', options=('Base', 'Contingency'), value='Base'), Output()), …

## 2.2 Operational  monetizacion 
$$Benefit = ES_c-EG_c-ENS_c$$

In [9]:
criticality = ACM(Cont_A.load_user,Cont_A.gen_data)

In [10]:
def Assessment_Performance(case):
    interact(Contingency_Assessment_Energy, day_name=day_list,case=fixed(case))

def Contingency_Assessment_Energy(day_name,case):
    if case == 'Base':
        hour_list = df_base_line[df_base_line.Day==day_name].Hour.unique()
        sec_assesment_by_day =  df_sec['cr_energy'][day_name]        
        interact(Energy_Assessment_by_hour, hour=hour_list,df_sec=fixed(sec_assesment_by_day)) 
    elif case == 'Contingency':
        asset_type_list = Cont_A.asset_list['Type'].unique()
        interact(Asset_Selection_E, asset_type=asset_type_list,day=fixed(day_name))    
def Asset_Selection_E(asset_type,day):
    asset_id_list                = Cont_A.asset_list[Cont_A.asset_list['Type']==asset_type].index.values
    interact(disc_asset_E, asset_id=asset_id_list,day=fixed(day))  
def disc_asset_E(asset_id,day):
    asset_fail                        = asset_id
    net_lf                            = Cont_A.Net_Configurarion_during_Contingency(Cont_A.net,asset_fail)
    df_base_C,df_load_C,sec_assesment = Cont_A.Run_Case_Load_growth(net_lf,load_growth,day=[day])
    hour_list = df_base_C[df_base_C.Day==day].Hour.unique()
    sec_assesment_by_day =  sec_assesment['cr_energy'][day]
    interact(Energy_Assessment_by_hour, hour=hour_list,df_sec=fixed(sec_assesment_by_day))  
def Energy_Assessment_by_hour(hour,df_sec):
    # Load
    df           = df_sec[hour]
    df_load      = df['Load']
    df_load_m    = df_load.melt(id_vars =['name']) 
    fig1         = px.sunburst(df_load_m, path=['variable','name'], values='value')
    df_cr_load   = criticality.Monetized_Energy_During_Contingency_by_hour(df_load)
    df_cr_load   = df_cr_load.reset_index()
    df_load      = df_cr_load.melt(id_vars =['name']) 
    fig3         = px.sunburst(df_load, path=['variable','name'], values='value')
    
    # Generation
    df_gen       = df['Gen']
    fig2         = px.sunburst(df_gen, path=['source','name'], values='gen')
    df_cr_gen    = criticality.Monetized_Gen_During_Contingency_by_hour(df_gen) 
    df_cr_gen    = df_cr_gen.reset_index()
    df_gen       = df_cr_gen.melt(id_vars =['name']) 
    fig4         = px.sunburst(df_gen, path=['variable','name'], values='value')    
    # ploting    
    fig = make_subplots(rows=2, cols=2, specs=[[{"type": "sunburst"}, {"type": "sunburst"}],[{"type": "sunburst"}, {"type": "sunburst"}]],
                       subplot_titles=("Demand - [MWh]", "Generation - [MWh]", "Demand Benefit -[$]", "Generation Benefit -[$]")
                       , column_widths=[1.0, 1.0],vertical_spacing=0.1)
    fig.add_trace(fig1.data[0], row=1, col=1)
    fig.add_trace(fig2.data[0], row=1, col=2)
    
    fig.add_trace(fig3.data[0], row=2, col=1)
    fig.add_trace(fig4.data[0], row=2, col=2)
    
    benefit      =  criticality.Benefit_During_Contingency(df_cr_load,df_cr_gen)
    l_text       = 'The benefit is =$ '+ str(round(benefit,2))
    fig.update_layout(height=750, width=950, title_text=l_text)
    
    fig.show()

In [18]:
interact(Assessment_Performance,case=l_case);

interactive(children=(Dropdown(description='case', options=('Base', 'Contingency'), value='Base'), Output()), …